In [1]:
shape_cat=""
vol=""

In [2]:
# Parameters
shape_cat = "mcal"
vol = "all"


In [3]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import fitsio as fio
import numpy as np
import ast

import astropy.cosmology
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from IPython.display import display, Math

In [4]:
if vol=="vlim":
    cluster_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17-vlim_lgt20_desformat_catalog.fit')[1].data)
    member_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17-vlim_lgt20_desformat_catalog_members.fit')[1].data)
    random_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3_wide+d10-mof-001d_run_redmapper_v6.4.17-2_randcat_z0.10-0.95_lgt020_vl50.fit')[1].data)
elif vol=="all":
    cluster_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog.fit')[1].data)
    member_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3-d10-mof-001d_run_redmapper_v6.4.17_lgt20_desformat_catalog_members.fit')[1].data)
    random_table=Table(fits.open('/lsst/troxel/y1a1/y1a1_gold_1.0.3_wide+d10-mof-001d_run_redmapper_v6.4.17-2_randcat_z0.10-0.95_lgt020_vl02.fit')[1].data)
else:
    raise TypeError("Wrong vol")
cluster_table.remove_columns(['P_C','C_LAMBDA','C_LAMBDA_ERR','Q_CEN']) #unknown columns

print("Number of members: ",len(member_table))
print("Number of clusters: ",len(cluster_table))
print("Number of members associated with multiple clusters: ",len(member_table['ID'])-len(np.unique(member_table['ID'])))

Number of members:  809723
Number of clusters:  16966
Number of members associated with multiple clusters:  5621


In [5]:
if shape_cat=="mcal":
    mcal = fio.FITS('/lsst/troxel/y1a1/mcal-y1a1-combined-riz-unblind-v4-matched.fits',memmap=True,mode='r')[1]
    print ("Metacal loaded")
    shape_id=mcal['coadd_objects_id'][:] 
elif shape_cat=="im3":
        mcal = fio.FITS('/lsst/troxel/y1a1/y1a1-im3shape_v5_unblind_v2_matched_v4.fits',memmap=True,mode='r')[1]
        print ("Metacal loaded")
        shape_id=mcal['coadd_objects_id'][:] 
else:
    raise 

Metacal loaded


In [6]:
print("Number of members without shape: ",len(member_table['ID'])-np.sum(np.in1d(member_table['ID'],shape_id)))

Number of members without shape:  3772


In [7]:
mcal.get_info()

{'hdunum': 2,
 'extnum': 1,
 'hdutype': 2,
 'extname': '',
 'hduname': '',
 'extver': 0,
 'hduver': 0,
 'is_compressed_image': 0,
 'header_start': 2880,
 'data_start': 17280,
 'data_end': 84075649920,
 'nrows': 136930995,
 'ncols': 80,
 'colinfo': [{'name': 'coadd_objects_id',
   'tform': 'K',
   'type': 81,
   'repeat': 1,
   'width': 8,
   'eqtype': 81,
   'tdim': [1],
   'tscale': 1.0,
   'tzero': 0.0},
  {'name': 'flags',
   'tform': 'J',
   'type': 41,
   'repeat': 1,
   'width': 4,
   'eqtype': 41,
   'tdim': [1],
   'tscale': 1.0,
   'tzero': 0.0},
  {'name': 'mask_frac',
   'tform': 'D',
   'type': 82,
   'repeat': 1,
   'width': 8,
   'eqtype': 82,
   'tdim': [1],
   'tscale': 1.0,
   'tzero': 0.0},
  {'name': 'psf_e1',
   'tform': 'D',
   'type': 82,
   'repeat': 1,
   'width': 8,
   'eqtype': 82,
   'tdim': [1],
   'tscale': 1.0,
   'tzero': 0.0},
  {'name': 'psf_e2',
   'tform': 'D',
   'type': 82,
   'repeat': 1,
   'width': 8,
   'eqtype': 82,
   'tdim': [1],
   'tscale':

In [8]:
problem_list =['MODEL_MAG', 'MODEL_MAGERR', 'MAG_LAMBDA_ERR', 'PZBINS',
               'PZ', 'RA_CENT', 'DEC_CENT', 'ID_CENT', 'LAMBDA_CHISQ_CENT',
               'ZLAMBDA_CENT', 'P_BCG', 'P_CEN', 'P_FG', 'P_SAT']

In [9]:
band_col_list = []
pz_col_list = []
alt_col_list =[]
for problem in problem_list:
    if cluster_table[problem].shape[1]==4:
        band_col_list.append(problem)
    elif cluster_table[problem].shape[1]==5:
        alt_col_list.append(problem)
    elif cluster_table[problem].shape[1]==21:
        pz_col_list.append(problem)
    else:
        print ("error")

In [10]:
band_list=['g','r','i','z','Y']
pz_index_list=['pz'+str(i+1) for i in range(21)]
alt_index_list=['Alt'+str(i+1) for i in range(5)]

In [11]:
for j in range(4):
    for i,column in enumerate(band_col_list):
        cluster_table[str(('Band',band_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(band_col_list)

In [12]:
for i,column in enumerate(pz_col_list):
    for j in range(21):
        cluster_table[str(('PZ',pz_index_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(pz_col_list)

In [13]:
for i,column in enumerate(alt_col_list):
    for j in range(5):
        cluster_table[str(('Alt',alt_index_list[j],column))]=cluster_table[column].data[:,j]
cluster_table.remove_columns(alt_col_list)

In [14]:
all_col_list=list(cluster_table.columns[0:54])
for i,column in enumerate(all_col_list):
    cluster_table[str(('All','All',column))]=cluster_table[column].data[:]
cluster_table.remove_columns(all_col_list)

In [15]:
for j in range(4):
    for i,column in enumerate(band_col_list[0:2]):
        member_table['('+band_list[j]+','+column+')']=member_table[column].data[:,j]
member_table.remove_columns(band_col_list[0:2]) 

In [16]:
clusters=cluster_table.to_pandas()
member=member_table.to_pandas()
random=random_table.to_pandas()
# center_table=None
# member_table=None

In [17]:
member_cols = member.columns[0:22]
shape_columns=pd.MultiIndex.from_product([['All'],list(mcal.get_colnames())],names=['Band','Data'])

In [18]:
center_bins=[m+str(n) for m,n in zip(['cen_']*5,np.arange(5)+1)]
center=pd.MultiIndex.from_product([['Center'],center_bins,member_cols])

In [19]:
cluster_band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['MODEL_MAG','MODEL_MAGERR','MAG_LAMBDA_ERR']],
                                     names=['band','data'])

In [20]:
# alt_bins=[m+str(n) for m,n in zip(['alt_']*5,np.arange(5)+1)]
# alt_columns=pd.MultiIndex.from_product([alt_bins,
#                                        alt_col_list.append(shape_names)],
#                                      names=['alt_index','data'])

In [21]:
member_band_columns=pd.MultiIndex.from_product([['g','r','i','z'],
                                       ['MODEL_MAG','MODEL_MAGERR',]],
                                     names=['Band','Data'])

In [22]:
shape_index=pd.MultiIndex.from_tuples([('MEM_MATCH_ID','TYPE')],names=['ClusterID','Type'])

In [23]:
random_columns=pd.MultiIndex.from_product([['All'],['RA', 'DEC', 'Z', 'LAMBDA_IN', 'AVG_LAMBDAOUT', 'SIGMA_LAMBDAOUT', 'WEIGHT']],
                                       names=['Band','Data'])

In [24]:
random.columns=random_columns

In [25]:
member_id=member['ID'].values

In [26]:
# for id in member_id:
#     mask = pd.Series(shape_id)==id

In [27]:
shape_id_series = pd.Series(shape_id)

In [28]:
def sort2n(x,y):
    """
    Sorts and matches two arrays of object ids where x is unique and y is not (in DES this is coadd_objects_id).
    Slower than sort2().
    """
    
    xsort = np.argsort(x)
    ysort = np.argsort(y)
    i_yx = np.sort(y[np.in1d(y, x, assume_unique=False)])
    i_x = xsort[x[xsort].searchsorted(i_yx)]
    i_y = ysort[y[ysort].searchsorted(i_yx)]
    
    return i_x, i_y

In [29]:
s2n_index=sort2n(member_id,shape_id)

In [30]:
print("The members that does not have shape measurement:{}".format(len(member_id)-len(s2n_index[0])))

The members that does not have shape measurement:9391


In [31]:
shape_half=pd.DataFrame(mcal.read(rows=[s2n_index[1]]).byteswap().newbyteorder())

In [32]:
member_half=member.iloc[s2n_index[0]]

In [33]:
member_half_index=pd.MultiIndex.from_tuples(list(zip(['All']*23,member_half.columns[0:23]))+list(member_band_columns),names=['Band','Data']) 

In [34]:
shape_half.columns=shape_columns
member_half.columns=member_half_index
shape_half.set_index(('All','coadd_objects_id'),inplace=True)
member_half.set_index(('All','ID'),inplace=True)

In [35]:
shapes=pd.concat([member_half,shape_half],axis=1,verify_integrity=True)
shapes.index.name="ID"

# im3 e correction

In [36]:
if shape_cat == "im3":
    shapes[('All','e1_old')]=shapes[('All','e1')]
    shapes[('All','e2_old')]=shapes[('All','e2')]
    shapes[('All','e1')]=shapes.apply(lambda x:x[('All','e1')]-x[('All','c1')],axis=1)
    shapes[('All','e2')]=shapes.apply(lambda x:x[('All','e2')]-x[('All','c2')],axis=1)

In [37]:
cluster_col_list=[ast.literal_eval(name) for i,name in enumerate(clusters.columns)]
cluster_col=pd.MultiIndex.from_tuples(cluster_col_list, names=['Type','Kind','Data'])

In [38]:
clusters.columns=cluster_col
clusters.set_index(('All', 'All', 'MEM_MATCH_ID'),inplace=True);

# Add PP

In [39]:
shapes[('All','PP')]=shapes.apply(lambda x:x[('All','P')]*x[('All','PFREE')],axis=1)
print ("Added PP column.")

Added PP column.


# Shape and PP masking

In [40]:
print("Members before flag_select mask: ",len(shapes))
shapes=shapes[shapes[('All', 'flags_select')]==0]
print("Members after flag_select mask: ",len(shapes))

Members before flag_select mask:  800332


Members after flag_select mask:  495540


In [41]:
print("Members before probability mask: ",len(shapes))
shapes=shapes[shapes[('All','PP')]>=0.55]
print("Members after probability mask: ",len(shapes))

Members before probability mask:  495540


Members after probability mask:  282735


# No center masking/ no cluster masking

In [42]:
# Drop clusters with no center
def drop_poor_centers(clusters_cat):
    print("The number of clusters before masking", len(clusters_cat))
    center_id=clusters_cat[('Alt', 'Alt1', 'ID_CENT')]
    p=shapes.loc[center_id][('All','P')].to_numpy()
    missing_mem_match_id=clusters_cat[np.isnan(p)].index
    cluster_masked=clusters_cat.drop(missing_mem_match_id)
    print("The number of clusters with no center shape data {}".format(np.sum(np.isnan(p))))
    print("The number of clusters after masking", len(cluster_masked))
    return(cluster_masked)

clusters=drop_poor_centers(clusters)

# Drop members with no cluster
def drop_poor_members(clusters_cat,shapes_cat):
    print("The number of members before masking: ",len(shapes_cat))
    valid_clusters_id=clusters_cat.index.to_numpy()
    shapes_match_id=shapes_cat[('All','MEM_MATCH_ID')].to_numpy()
    valid_members=np.isin(shapes_match_id,valid_clusters_id)
    print("The number of members without a cluster: ",np.sum(~valid_members))
    members_masked=shapes_cat[valid_members]
    print("Number of galaxies after masking: {}".format(len(members_masked)))
    return(members_masked)
    
shapes=drop_poor_members(clusters,shapes)

The number of clusters before masking 16966
The number of clusters with no center shape data 4483
The number of clusters after masking 12483
The number of members before masking:  282735


The number of members without a cluster:  79655
Number of galaxies after masking: 203080


/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """


In [43]:
clusters=drop_poor_centers(clusters)
shapes=drop_poor_members(clusters,shapes)

The number of clusters before masking 12483
The number of clusters with no center shape data 1
The number of clusters after masking 12482
The number of members before masking:  203080
The number of members without a cluster:  10
Number of galaxies after masking: 203070


/home/cz136/anaconda3/envs/balrog/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """


# Add $\phi$

In [44]:
def get_theta_for_obj(obj1,obj2):
    ra1,dec1=(obj1[('All', 'RA')],obj1[('All', 'DEC')])
    ra2,dec2=(obj2[('All', 'RA')],obj2[('All', 'DEC')])
    
    
    
    c1=SkyCoord(ra1*u.deg,dec1*u.deg)
    c2=SkyCoord(ra2*u.deg,dec2*u.deg)
    
    
    return(90-c1.position_angle(c2).degree)

In [45]:
def get_rel_theta(member):
    cluster=clusters.loc[member[('All','MEM_MATCH_ID')]]
    center=shapes.loc[cluster[('Alt','Alt1','ID_CENT')]]
    theta=get_theta_for_obj(center,member)
    return(theta)

In [46]:
def get_pa_for_member(obj):
    e1=obj['All']['e1']
    e2=obj['All']['e2']
    
    alpha =0.5*np.arctan2(-e2,-e1)*180/np.pi
    if alpha>=0:
        alpha=(alpha-90)
    elif alpha<0:
        alpha=90+alpha
    else:
        raise 
    return(alpha)

In [47]:
shapes[('All','alpha')]=shapes.apply(lambda x:get_pa_for_member(x),axis=1)

In [48]:
error_list_phi=[]
error_list_e=[]

def get_phi_for_member(member):
    
    """Input: cluster series
        Output: an array of position angle weighted by PP
        Record KeyError in an array (usually caused by uncomplete shape catalog).
    """
    
    cluster_id=member[('All','MEM_MATCH_ID')]
    cluster=clusters.loc[cluster_id]
    center_id=cluster[('Alt', 'Alt1', 'ID_CENT')]
    center = shapes.loc[center_id]

    pa=member[('All','alpha')]
    theta= get_theta_for_obj(member,center)
    phi=(pa-theta)
    #     print(center.index)
    return(phi)    

In [49]:
shapes[('All','phi')]=shapes.apply(lambda x:get_phi_for_member(x),axis=1)

In [50]:
def process_phi(phi_1d):

    phi_1d=np.where(phi_1d<=0,-phi_1d,phi_1d)
    phi_1d=np.where(phi_1d>=180,phi_1d-180,phi_1d)
    phi_1d=np.where(phi_1d>=90,180-phi_1d,phi_1d)

    
    return(phi_1d)

In [51]:
shapes[('All','phi')]=process_phi(shapes[('All','phi')])

# Add theta

In [52]:
shapes[('All','Theta_cen')]=shapes.apply(lambda x:get_rel_theta(x),axis=1)

In [53]:
shapes[('All','Theta_cen')]=shapes.apply(lambda x:360-x[('All','Theta_cen')] if x[('All','Theta_cen')]>=180 else x[('All','Theta_cen')],axis=1)
shapes[('All','Theta_cen')]=shapes.apply(lambda x:360+x[('All','Theta_cen')] if x[('All','Theta_cen')]<=-180 else x[('All','Theta_cen')],axis=1)
shapes[('All','Theta_sat')]=shapes.apply(lambda x:-(180-x[('All','Theta_cen')]) if x[('All','Theta_cen')]>=0 else (180+x[('All','Theta_cen')]),axis=1)

# Add angR

In [54]:
H0=100 #km s-1 Mpc-1
Om0=0.3
Ode0=0.7
lcdm=astropy.cosmology.FlatLambdaCDM(H0, Om0)

In [55]:
shapes[('All','angR')]=shapes.apply(lambda x:np.float(lcdm.angular_diameter_distance(x[('All','Z')])/u.Mpc),axis=1)
random[('All','angR')]=random.apply(lambda x:np.float(lcdm.angular_diameter_distance(x[('All','Z')])/u.Mpc),axis=1)

# Add e

In [56]:
def get_e(sat):
    cluster_id=sat[('All','MEM_MATCH_ID')]
    cluster=clusters.loc[cluster_id]
    center_id=cluster[('Alt', 'Alt1', 'ID_CENT')]
    cen = shapes.loc[center_id]
    
    e=np.sqrt(sat[('All','e1')]**2+sat[('All','e2')]**2)
    delta=get_theta_for_obj(cen,sat)
    alpha=sat[('All','alpha')]
    ep,ex=(e*np.cos(2*(delta-alpha)),e*np.sin(2*(delta-alpha)))
    return(ep,ex,e)

In [57]:
shapes[('All','e')]=shapes.apply(lambda x:get_e(x)[2],axis=1)
shapes[('All','ep')]=shapes.apply(lambda x:get_e(x)[0],axis=1)
shapes[('All','ex')]=shapes.apply(lambda x:get_e(x)[1],axis=1)

# Calculate cluster elipticity

In [58]:
display(Math(r"$M_{\mathrm{xx}} \equiv\left\langle\frac{x^{2}}{r^{2}}\right\rangle=\frac{\sum_{i} p_{\mathrm{mem}, i} \frac{x_{i}^{2}}{r_{i}^{2}}}{\sum_{i} p_{\mathrm{mem}, i}}$"))

<IPython.core.display.Math object>

In [59]:
def get_Q_U(cluster):
    
    mem_match_id=cluster.name
    members=shapes[shapes['All','MEM_MATCH_ID']==mem_match_id]
    
    members=members[members[('All','PP')]>=0.2]
    
    def get_M_for_cluster():
        p = members[('All','PP')]
        r = members[('All','R')]
        pa = members[('All','Theta_cen')]
        x = r * np.cos(pa)
        y = r * np.sin(pa)
        
        def mom_2(p,x,y,r):
            return(np.sum(p*x*y/r**2)/np.sum(p))
        
        Mxx=mom_2(p,x,x,r)
        Mxy=mom_2(p,x,y,r)
        Myy=mom_2(p,y,y,r)
        return(Mxx,Mxy,Myy)

    Mxx,Mxy,Myy=get_M_for_cluster()
    Q,U=Mxx-Myy,2*Mxy
    
    return (Q,U)


def get_pa_from_Q_U(Q,U):
    α =0.5*np.arctan2(U,Q)*180/np.pi
    return(α)

In [60]:
clusters[('All','All','Q')]=clusters.apply(lambda x:get_Q_U(x)[0],axis=1)
clusters[('All','All','U')]=clusters.apply(lambda x:get_Q_U(x)[1],axis=1)
clusters[('All','All','Beta')]=clusters.apply(lambda x:get_pa_from_Q_U(x[('All','All','Q')],x[('All','All','U')]),axis=1)
clusters[('All','All','Ce')]=clusters.apply(lambda x:np.sqrt(x[('All','All','U')]**2+x[('All','All','U')]**2),axis=1)

In [61]:
def get_normLR(shape):
    cluster_id=shape[('All','MEM_MATCH_ID')]
    LR = clusters.loc[cluster_id][('All','All','R_LAMBDA')]
    angR=shape[('All','angR')]
    normR=angR/LR
    return(normR)

In [62]:
shapes[('All','normLR')]=shapes.apply(lambda x:get_normLR(x),axis=1)

In [63]:
def add_r_lambda(shape):
    cluster_id=shape[('All','MEM_MATCH_ID')]
    LR = clusters.loc[cluster_id][('All','All','R_LAMBDA')]
    return(LR)

In [64]:
shapes[('All','R_LAMBDA')]=shapes.apply(lambda x:add_r_lambda(x),axis=1)

# Save

In [65]:
clusters.to_pickle('/home/cz136/project/sa/data/{}/{}/clusters.pkl'.format(shape_cat,vol))

In [66]:
shapes.to_pickle('/home/cz136/project/sa/data/{}/{}/shapes.pkl'.format(shape_cat,vol))

In [67]:
random.to_pickle('/home/cz136/project/sa/data/{}/{}/random.pkl'.format(shape_cat,vol))

## z masking

In [68]:
def drop_shapes_wo_cluster(clusters,shapes):
    valid_clusters_id=clusters.index.to_numpy()
    shapes_match_id=shapes[('All','MEM_MATCH_ID')].to_numpy()
    print("Number of galaxies before masking: {}".format(len(shapes_match_id)))
    valid_members=np.isin(shapes_match_id,valid_clusters_id)
    shapes=shapes[valid_members]
    return(shapes)

In [69]:
high_z=clusters[clusters['All','All','Z_LAMBDA']>=0.35]
low_z=clusters[(clusters['All','All','Z_LAMBDA']<=0.35) & (clusters['All','All','Z_LAMBDA']>=0.1)]

In [70]:
high_z_shapes=drop_shapes_wo_cluster(high_z,shapes)
low_z_shapes=drop_shapes_wo_cluster(low_z,shapes)

Number of galaxies before masking: 203070


Number of galaxies before masking: 203070


In [71]:
high_z.to_pickle('/home/cz136/project/sa/data/{}/{}/clusters_high_z.pkl'.format(shape_cat,vol))
low_z.to_pickle('/home/cz136/project/sa/data/{}/{}/clusters_low_z.pkl'.format(shape_cat,vol))
high_z_shapes.to_pickle('/home/cz136/project/sa/data/{}/{}/shapes_high_z.pkl'.format(shape_cat,vol))
low_z_shapes.to_pickle('/home/cz136/project/sa/data/{}/{}/shapes_low_z.pkl'.format(shape_cat,vol))